In [1]:
import sys
import os


sys.path.append(os.path.abspath("../project"))  

In [15]:
import cirq
from qudit_cirq.circuit_builder import create_circuit
from qudit_cirq.utils import format_out, printVector, tensor_product, add_measurements
from qudit_cirq.qudit import quditXGate, quditZGate, quditHGate, quditCNOTGate, state_vector
import numpy as np

In [16]:
d = 3

In [27]:
circuit, qudits, qudit_order = create_circuit(
    d,
    (quditHGate, 'q0'),    
    (quditXGate, 'q0'),    
    (quditZGate, 'q0'),    
    (quditCNOTGate, ['q0', 'q1'])  
)

In [28]:
circuit

q0 (d=3): ───H(d=3)───X(d=3)───Z(d=3)───┬───────────
                                        │
q1 (d=3): ──────────────────────────────CNOT(d=3)───

In [29]:
add_measurements(circuit, qudit_order)

In [30]:
circuit

q0 (d=3): ───H(d=3)───X(d=3)───Z(d=3)───┬───────────M('meas_q0')───
                                        │
q1 (d=3): ──────────────────────────────CNOT(d=3)───M('meas_q1')───

In [31]:
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=165)

In [32]:
for qudit in qudit_order:
    key = f"meas_{qudit.name}"
    hist = result.histogram(key=key)
    print(f"\nMeasurement results for {qudit.name}:")
    for outcome, count in sorted(hist.items()):
        probability = count / 1000
        print(f"State |{outcome}>: {probability:.2f}")


Measurement results for q0:
State |0>: 0.05
State |1>: 0.12

Measurement results for q1:
State |0>: 0.05
State |1>: 0.12


In [84]:
d = 3 

In [85]:
qudit0 = cirq.NamedQid('q0', dimension=d)  
qudit1 = cirq.NamedQid('q1', dimension=d)  
qudit2 = cirq.NamedQid('q2', dimension=d) 
qudit_order = [qudit0, qudit1, qudit2]

circuit = cirq.Circuit()

In [86]:
initial_state_q0 = np.array([0, 1, 0], dtype=complex)

In [87]:
initial_state = np.kron(
    np.kron(initial_state_q0, [1, 0, 0]), 
    [1, 0, 0]              
)

In [88]:
circuit.append([
    quditHGate(d)(qudit1),
    quditCNOTGate(d)(qudit1, qudit2)
])


circuit.append([
    quditCNOTGate(d)(qudit0, qudit1),
    quditHGate(d)(qudit0)
])


circuit.append([
    cirq.measure(qudit0, key='m0'),
    cirq.measure(qudit1, key='m1')
])

In [89]:
print("Teleportation Circuit:")
print(circuit)

Teleportation Circuit:
q0 (d=3): ────────────────────────┬───────────H(d=3)────M('m0')───
                                  │
q1 (d=3): ───H(d=3)───┬───────────CNOT(d=3)───M('m1')─────────────
                      │
q2 (d=3): ────────────CNOT(d=3)───────────────────────────────────


In [90]:
initial_state

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j])

In [96]:
simulator = cirq.Simulator()
result = simulator.simulate(circuit, initial_state=initial_state)

In [97]:
m0 = int(result.measurements['m0'].item())
m1 = int(result.measurements['m1'].item())
print(f"\nAlice's measurement results: m0 = {m0}, m1 = {m1}")


Alice's measurement results: m0 = 0, m1 = 2


In [98]:
corrected_state_vector = result.final_state_vector

final_state = corrected_state_vector.reshape((d, d, d))

state_qudit2 = np.sum(np.sum(final_state, axis=0), axis=0)

state_qudit2 = state_qudit2 / np.linalg.norm(state_qudit2)

In [99]:
state_qudit2

array([0.+0.j, 1.+0.j, 0.+0.j], dtype=complex64)

In [100]:
print("\nInitial state of qudit0:")
for i in range(d):
    amplitude = initial_state_q0[i]
    print(f"|{i}>: amplitude = {amplitude}")

if np.allclose(state_qudit2, initial_state_q0):
    print("\nTeleportation successful: The state of qudit2 matches the initial state of qudit0.")
else:
    print("\nTeleportation failed: The state of qudit2 does not match the initial state of qudit0.")


Initial state of qudit0:
|0>: amplitude = 0j
|1>: amplitude = (1+0j)
|2>: amplitude = 0j

Teleportation successful: The state of qudit2 matches the initial state of qudit0.
